# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data
data_dir = 'data/'
summary_dir = data_dir+'vgg/summary/'
training_file = data_dir + 'train.p'
validation_file= data_dir + 'valid.p'
testing_file = data_dir + 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
import numpy as np

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_validation = X_valid.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).size

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import random
# Visualizations will be shown in the notebook.
%matplotlib inline


In [ ]:
def show_class_images(y_class, num_images=1):
    class_images = X_train[y_train==y_class]
    idx = np.random.randint(class_images.shape[0], size=num_images)
    class_sample = class_images[idx]
    
    for image in class_sample:
        plt.figure()
        plt.imshow(image)

show_class_images(30, 2)

In [ ]:
y_unique, y_counts = np.unique(y_train, return_counts=True)
# plt.hist(y_counts, normed=True, bins=y_unique.size)
plt.barh(y_unique, y_counts)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

#### Option 1: Simple Normalization

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
def normalize_images(images):
    image_norm = (images.astype(np.float32) - 128) / 128
    print('Mean: ', np.mean(image_norm))
    return image_norm

X_train_norm = normalize_images(X_train)
X_valid_norm = normalize_images(X_valid)
X_test_norm = normalize_images(X_test)

#### Option 2: Mean Subtraction + Normalization - Seems to converge slower than option 1

In [ ]:
x_mean = np.mean(X_train, axis=(0, 1, 2))
x_std = np.std(X_train, axis=(0, 1, 2))
X_train_norm = (X_train - x_mean) / x_std
X_valid_norm = (X_valid - x_mean) / x_std
X_test_norm = (X_test - x_mean) / x_std

### Model Architecture

In [ ]:
import tensorflow as tf
from sklearn.utils import shuffle

### Initialize Variables

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(shape=(None,)+image_shape, dtype=tf.float32, name='x')
y = tf.placeholder(shape=(None), dtype=tf.int32, name='y')
y_hot = tf.one_hot(y, n_classes, name='y_hot')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')


num_examples = len(X_train)
BATCH_SIZE = 256

global_step = tf.Variable(0, name='global_step', trainable=False)
learning_rate_start = 0.001
learning_rate = tf.train.exponential_decay(learning_rate_start, global_step, int(num_examples/BATCH_SIZE)*2, .95, staircase=True, name='exponential_learning_rate')

# Initial size (32 x 32 x 3)
img_w, img_h, img_d = image_shape

In [ ]:
def input_summaries(x, y, name):
    with tf.name_scope('input_summary'):
        with tf.name_scope(name):
            mean = tf.reduce_mean(x)
            stddev = tf.sqrt(tf.reduce_mean(tf.square(x - mean)))
            tf.summary.scalar('mean', mean)
            tf.summary.scalar('stddev', stddev)
            tf.summary.histogram('y_histogram', y)

input_summaries(x, y, 'data_train')

### VGG16 like model

In [ ]:
def conv_layer(input_layer, layer_name, k_depth, filter_size=3, stride=1, padding='SAME'):
    with tf.variable_scope(layer_name):
        input_depth = input_layer.shape[-1]
        conv_strides = (1, stride, stride, 1)
        filter_shape = (filter_size, filter_size, input_depth, k_depth)
        
        conv_w = tf.get_variable('W', shape=filter_shape, initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv_b = tf.Variable(tf.zeros(k_depth), name='B')
        with tf.variable_scope('Z'):
            conv = tf.nn.conv2d(input=input_layer, filter=conv_w, strides=conv_strides, padding=padding) + conv_b
        conv_relu = tf.nn.relu(conv, 'A')
        
        return conv_relu
    
def conv_block(input_layer, block_name, k_depth):
    with tf.variable_scope(block_name):
        conv1 = conv_layer(input_layer, layer_name='Conv1', k_depth=k_depth)
        batch1 = tf.layers.batch_normalization(conv1)
        conv2 = conv_layer(batch1, layer_name='Conv2', k_depth=k_depth)
        pool = tf.nn.max_pool(conv2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='VALID', name='Pool')
    return pool
    

def VGG16(input_x, keep_prob):
    # 32 x 32 x 3
    
    # Convolution1 Layer 1
    # 32 x 32 x 64
    conv_block1 = conv_block(input_x, block_name='ConvBlock1', k_depth=32)
    
    batch_norm_conv1 = tf.layers.batch_normalization(conv_block1, name='BatchNormConvBlock1')
    
    dropout_conv1 = tf.nn.dropout(batch_norm_conv1, keep_prob, name='DropoutConv1')

    # 16 x 16 x 128
    conv_block2 = conv_block(dropout_conv1, block_name='ConvBlock2', k_depth=64)
    
    batch_norm_conv2 = tf.layers.batch_normalization(conv_block2, name='BatchNormConvBlock2')

    # Dense Layers
    # 5 x 5 x 16 = 400
    flatten = tf.contrib.layers.flatten(batch_norm_conv2, scope="Flatten")

    batch_norm_flatten = tf.layers.batch_normalization(flatten, name='BatchNormFlatten')
    
    # 400 x 120
    dense1 = tf.layers.dense(batch_norm_flatten, units=256, activation=tf.nn.relu, name="Dense1")
    
    batch_norm_dense1 = tf.layers.batch_normalization(dense1, name='BatchNormDense1')
    
    dropout_dense1 = tf.nn.dropout(batch_norm_dense1, keep_prob, name='DropoutDense1')

    # 120 x 84
    dense2 = tf.layers.dense(dropout_dense1, units=512, activation=tf.nn.relu, name="Dense2")
    
    batch_norm_dense2 = tf.layers.batch_normalization(dense2, name='BatchNormDense2')
    
    with tf.name_scope('dropout'):
        dropout_dense2 = tf.nn.dropout(batch_norm_dense2, keep_prob, name='DropoutDense2')
        tf.summary.scalar('dropout_keep_probability', keep_prob)

    # 84 x 43
    dense3 = tf.layers.dense(dropout_dense2, units=n_classes, name="Output")
    
    return dense3

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

logits = VGG16(x, keep_prob)
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_hot, logits=logits)
    with tf.name_scope('total'):
        loss = tf.reduce_mean(cross_entropy)
tf.summary.scalar('cross_entropy_loss', loss)

with tf.name_scope('training'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    tf.summary.scalar('learning_rate', learning_rate)
    training_operation = optimizer.minimize(loss, global_step=global_step)
    
def train_batch(x_data, y_data, batch_size, writer=None):
    num_examples = len(x_data)
    for offset in range(0, num_examples, batch_size):
        end = offset + batch_size
        batch_x, batch_y = x_data[offset:end], y_data[offset:end]
        acc, summary = sess.run([training_operation, summaries_merged], feed_dict={x: batch_x, y: batch_y, keep_prob: 0.35})
        if writer is not None:
            writer.add_summary(summary, global_step.eval())
    

### Validation/accuracy operations

In [ ]:
with tf.name_scope('accuracy'):
    predictions = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y_hot, 1))
    accuracy_operation = tf.reduce_mean(tf.cast(predictions, tf.float32))
    tf.summary.scalar('accuracy', accuracy_operation)

def evaluate_accuracy(writer=None):
    va = get_accuracy(X_valid_norm, y_valid, BATCH_SIZE)
    ta = get_accuracy(X_train_norm, y_train, BATCH_SIZE)
    print('Validation accuracy:', va)
    print('Training accuracy:', ta)
    if writer is not None:
        acc_summary = tf.Summary()
        acc_summary.value.add(tag="validation_accuracy", simple_value=va)
        acc_summary.value.add(tag="training_accuracy", simple_value=ta)
        train_writer.add_summary(acc_summary, global_step.eval())

def get_accuracy(x_data, y_data, batch_size):
    total_accuracy = 0
    accuracies = []
    sess = tf.get_default_session()
    num_examples = len(x_data)
    for offset in range(0, num_examples, batch_size):
        end = offset+BATCH_SIZE
        batch_x, batch_y = x_data[offset:end], y_data[offset:end]
        feed_dict = {x: batch_x, y: batch_y, keep_prob: 1};
        accuracy = sess.run(accuracy_operation, feed_dict=feed_dict)
        accuracies.append(accuracy)
    avg_accuracy = np.mean(accuracies)
    return avg_accuracy
    

## Running the Model

In [ ]:
num_epochs = 10
summaries_merged = tf.summary.merge_all()
saver = tf.train.Saver()

In [ ]:
# Summary stuff
train_writer = tf.summary.FileWriter(summary_dir+'train')
valid_writer = tf.summary.FileWriter(summary_dir+'validation')
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    train_writer.add_graph(sess.graph)
    num_examples = len(X_train)
    
    print('Training...')
    for epoch in range(num_epochs):
        shuffle_x, shuffle_y = shuffle(X_train_norm, y_train)
        
        print('EPOCH:', epoch+1)
        train_batch(shuffle_x, shuffle_y, BATCH_SIZE, train_writer)
            
        evaluate_accuracy(train_writer)
        
    save_path = saver.save(sess, data_dir+'tmp/model.ckpt')
    print('Model saved to path:', save_path)

### More Epochs

In [ ]:
num_epochs = 10

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, data_dir+"tmp/model.ckpt")
    print('Model restored')
    num_examples = len(X_train)
    
    print('Training...')
    for epoch in range(num_epochs):
        shuffle_x, shuffle_y = shuffle(X_train_norm, y_train)
        
        print('EPOCH:', epoch+1)
        train_batch(shuffle_x, shuffle_y, BATCH_SIZE)
        
        evaluate_accuracy()
        
    save_path = saver.save(sess, data_dir+'tmp/model_train_2.ckpt')

### Test model accuracy on Test Set

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, data_dir+"tmp/model.ckpt")
    print('Testing...')
    acc = get_accuracy(X_test_norm, y_test, BATCH_SIZE)
    print('Testing accuracy:', acc)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
import urllib
external_image_dir = data_dir + 'external_images/'

#### Download image urls into image dir

In [ ]:
# Keep Right - 38
urllib.request.urlretrieve("https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Keep_right_Portugal_20100107.jpg/170px-Keep_right_Portugal_20100107.jpg", external_image_dir+"38.jpg")

# Roundabout mandatory - 40
urllib.request.urlretrieve("https://ak6.picdn.net/shutterstock/videos/19542826/thumb/9.jpg?i10c=img.resize(height:160)", external_image_dir+"40.jpg")

# Beware Ice/Snow - 30
urllib.request.urlretrieve("http://l7.alamy.com/zooms/b829800d974a4a4fb09389094471d6e4/snowflake-triangular-warning-sign-wales-warning-of-ice-welsh-bilingual-exb9f1.jpg", external_image_dir+"30.jpg")

# No Vehicles - 15
urllib.request.urlretrieve("http://2.bp.blogspot.com/-rAbhCLGEUwg/Tf8yNSMTJ-I/AAAAAAAABdE/f83W3DT04Ko/s1600/DSC02651.JPG", external_image_dir+"15.jpg")

# Priority road - 12
urllib.request.urlretrieve("http://2.bp.blogspot.com/-yeLDfO35Mws/UXVosStqjII/AAAAAAAALTo/y72zdixqHT4/s1600/euro_sign_graffiti_priority_road_germany_sm.JPG", external_image_dir+"12.jpg")

# Bumpy road - 22
urllib.request.urlretrieve("https://image.shutterstock.com/z/stock-photo-german-traffic-sign-bumpy-road-for-meters-415459579.jpg", external_image_dir+"22.jpg")

# Road work - 25
urllib.request.urlretrieve("http://www.springdot.org/baustelle/01/01.jpg", external_image_dir+"25.jpg")

# Speed limit - 1
urllib.request.urlretrieve("http://c8.alamy.com/comp/G667W0/road-sign-speed-limit-30-kmh-zone-passau-bavaria-germany-G667W0.jpg", external_image_dir+"1.jpg")

#### Load images and labels

In [ ]:
import os
import matplotlib.image as mpimg
import os.path

image_files = os.listdir(external_image_dir)
images = [mpimg.imread(external_image_dir+file) for file in image_files]
labels = np.array([int(os.path.splitext(file)[0]) for file in image_files])

In [ ]:
import scipy

def resize_images(images, size):
    resized = [scipy.misc.imresize(img, size) for img in images]
    return np.array(resized)

In [ ]:
for image in images:
    plt.figure()
    plt.imshow(image)


In [ ]:
with tf.name_scope('prediction'):
    prediction_operation = tf.nn.softmax(logits=logits)

In [ ]:
def predict_images(images):
    resized_images = resize_images(images, [32, 32])
    with tf.Session() as sess:
        saver.restore(sess, data_dir+"tmp/model2.ckpt")
        feed_dict = {x: resized_images, keep_prob: 1};
        predictions = sess.run(prediction_operation, feed_dict=feed_dict)
        return np.argmax(predictions, axis=1)

### Predict the Sign Type for Each Image

In [ ]:
predictions = predict_images(images)

print('Predictions:', predictions)
print('Actual label:', labels)

### Analyze Performance

In [ ]:
accuracy = np.mean(predictions == labels)
print('Accuracy:', accuracy)

### Test model on cropped images

In [ ]:
# Cropping images by hand to get better view of sign
cropped_images = np.copy(images)
cropped_images[0] = images[0][20:220, 20:220]
cropped_images[1] = images[1][30:90, 55:115]
cropped_images[2] = images[2][:300, :]
cropped_images[3] = images[3][50:-100, 250:1050]
cropped_images[4] = images[4][25:425, 100:500]
cropped_images[5] = images[5][220:600, 550:930]
cropped_images[6] = images[6][50:660, 450:1050]


In [ ]:
for image in cropped_images:
    plt.figure()
    plt.imshow(image)


In [ ]:
crop_predictions = predict_images(cropped_images)

print('Crop Predictions:', crop_predictions)
print('Actual label:', labels)
accuracy = np.mean(crop_predictions == labels)
print('Accuracy:', accuracy)

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
with tf.name_scope('top_k'):
    top_k_operation = tf.nn.top_k(prediction_operation, k=3)
    
def top_k_images(images):
    resized_images = resize_images(images, [32, 32])
    with tf.Session() as sess:
        saver.restore(sess, data_dir+"tmp/model2.ckpt")
        feed_dict = {x: resized_images, keep_prob: 1};
        return sess.run(top_k_operation, feed_dict=feed_dict)
    
print(top_k_images(cropped_images))
print(labels)

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input, keep_prob : 1})
    featuremaps = activation.shape[3]
    print('Featuremaps:', featuremaps)
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(8,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
X_train.shape
tf.constant(X_train[0]).shape
img = tf.expand_dims(X_train[0], 0)
img.shape

In [ ]:
# test = tf.get_default_graph().get_tensor_by_name("ConvBlock1/Conv1/W")
# op = tf.get_default_graph().get_operation_by_name("BatchNormFlatten")

with tf.Session() as sess:
    saver.restore(sess, data_dir+"tmp/model2.ckpt")
    
#     for v in tf.get_default_graph().as_graph_def().node:
#         print (v.name)

#     var_23 = [print(v) for v in tf.global_variables()]
    activation = tf.get_default_graph().get_tensor_by_name("ConvBlock1/Conv1/A:0")
    activation = tf.get_default_graph().get_tensor_by_name("ConvBlock1/Conv2/A:0")
    activation = tf.get_default_graph().get_tensor_by_name("ConvBlock2/Conv1/A:0")
    activation = tf.get_default_graph().get_tensor_by_name("ConvBlock2/Conv2/A:0")
    img = tf.expand_dims(X_train[0], 0)
    img = np.array([X_train[0]])
    outputFeatureMap(img, activation)


In [ ]:
outputFeatureMap(X_train[0], )